In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
import sys
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import glob
# np행렬 생략 부분 출력 펼쳐서 보기 옵션
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

In [ ]:
# GPU메모리 증가 허용하기 코드..내가 알아야할 필요가 있을지..? 뭔 이야기인지는 모르겠음.
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

class fashion_tools(object):
    def __init__(self,imageid,model,version=1.1):
        self.imageid = imageid
        self.model   = model
        self.version = version
        
    def get_dress(self,stack=False):
        """limited to top wear and full body dresses (wild and studio working)"""
        """takes input rgb----> return PNG"""
        name =  self.imageid
        file = name
        # file = cv2.imread(name)
        file = tf.image.resize_with_pad(file,target_height=512,target_width=512)
        rgb  = file.numpy()
        file = np.expand_dims(file,axis=0)/ 255.
        seq = self.model.predict(file)
        seq = seq[3][0,:,:,0]
        seq = np.expand_dims(seq,axis=-1)
        c1x = rgb*seq
        c2x = rgb*(1-seq)
        cfx = c1x+c2x
        dummy = np.ones((rgb.shape[0],rgb.shape[1],1))
        rgbx = np.concatenate((rgb,dummy*255),axis=-1)
        rgbs = np.concatenate((cfx,seq*255.),axis=-1)
        if stack:
            stacked = np.hstack((rgbx,rgbs))
            return stacked
        else:
            return rgbs
        
    def get_patch(self):
        return None

    def development(self):
        n_ = '\n'
        return (f"VERSION : {self.version} {n_} Interesting tools to be added to the workflow pipe!!!")

# 이미지 패딩값 주는 코드
def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    # print(img.size)
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)

# 결과 값 이미지 자르기
def crop(image):
    image_crop = image[:,512:]
    image_crop = np.uint8(image_crop)
    return black(image_crop)

# 투명값 검정배경 입혀주기.
def black(image_crop):
    for i in image_crop:
        for j in i:
            if j[3] < 80:
                j[0] = 0
                j[1] = 0
                j[2] = 0
    image_black = image_crop
    # cv2_imshow(image_black)
    return resize(image_black)

# 리사이즈 인풋값 맞춰 주기.
def resize(image_black):
    image_resize = cv2.resize(image_black, (28,28))
    return gray(image_resize)

# 그레이 스케일.. 후 픽셀 출력..
def gray(image_resize):
    image_gray = cv2.cvtColor(image_resize, cv2.COLOR_BGRA2GRAY)
    return image_gray


if __name__ == "__main__":
    sample_images_path = glob.glob("/content/drive/My Drive/Colab Notebooks/blazer_images_choice/*.jpg")
    # 훈련된 모델 가지고 오는 경로 설정.
    saved = load_model("/content/drive/My Drive/Colab Notebooks/데이터/topwears.h5")

    for idx, image_path in enumerate(sample_images_path):
        img = Image.open(image_path)
        img = resize_with_padding(img, (300, 300))
        img = np.array(img)
    
        ###running code
        api    = fashion_tools(img,saved)
        image = api.get_dress(True)
        result_image = crop(image)

        # 파일저장.
        cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/preprocessing_images/preprocessing_blazer_images/prepro_blazer_image{idx}.jpg', result_image)
        if idx % 50 == 0:
            print(idx,'done')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0 done
10 done
20 done
30 done
40 done
50 done
60 done
70 done
80 done
90 done
100 done
110 done
120 done
130 done
140 done
150 done
160 done
170 done
180 done
190 done
200 done
210 done
220 done
230 done
240 done
250 done
260 done
270 done
280 done
290 done
300 done
310 done
320 done


ValueError: ignored